In [ ]:
import requests
import pandas as pd

# API key:
api_key = 'ff19d2af10bc82cd94f3a9437f7be99865b71ef9'

# List years and months
years = range(2010, 2025)
months = ["jan", "feb", "mar", "apr", "may", "jun", "jul", "aug", "sep", "oct", "nov", "dec"]

# Define the base part of the URL for the Census API
base_url_template = "https://api.census.gov/data/{year}/cps/basic/{month}"

# Common variables to retrieve, excluding (GT)CBSA variables
common_vars = "PRTAGE,PTDTRACE,PEEDUCA,PEMLR,PESEX,HEFAMINC,PEMARITL,HRYEAR4,HRMONTH"

# Define the (GT)CBSA codes for the cities
cities = {
    "New York": "35620",
    "Chicago": "16980",
    "Minneapolis": "33460",
    "Atlanta": "12060",
    "Houston": "26420",
    "Seattle": "42660",
    "Phoenix": "38060",
    "Miami": "33100"
}

all_data = pd.DataFrame()

for year in years:
    for month in months:
        # Determine the correct city code variable based on the year
        city_var = "CBSA" if year == 2024 else "GTCBSA"

        # Format the URL for the specific year and month
        base_url = base_url_template.format(year=year, month=month)

        # Format the variables to include the correct city code variable
        variables = f"{common_vars},{city_var}"

        params = {
            "get": variables,
            "key": api_key
        }

        # Make the actual API request
        response = requests.get(base_url, params=params)
        try:
            data = response.json()
            df = pd.DataFrame(data[1:], columns=data[0])

            # Rename the city variable to "CityCode"
            df.rename(columns={city_var: "CityCode"}, inplace=True)

            # Filter the data for the  cities
            df_filtered = df[df['CityCode'].isin(cities.values())].copy()

            # Add a city name column
            df_filtered.loc[:, 'City'] = df_filtered['CityCode'].map({v: k for k, v in cities.items()})

            # Append to the all_data DataFrame
            all_data = pd.concat([all_data, df_filtered], ignore_index=True)

            # Print success message
            print(f"Successfully retrieved data for {year}-{month}")

        except ValueError:
            print(f"Failed to retrieve data for {year}-{month}")

# Save the collected data to a CSV file
all_data.to_csv('CPS.Demographic.Data.Semester.Project.csv', index=False)

print("Data retrieval complete. Check the 'CPS.Demographic.Data.Semester.Project.csv' file.")